> ### A researcher is	interested in how variables, such as GRE (Graduate	Record	Exam scores), GPA	(grade	point	average) and prestge of	the	undergraduate institution, effect admission	into graduate school. The	response variable,admit/don’t admit, is a binary variable	

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [5]:
admission_data = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

In [6]:
admission_data.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
